In [2]:
#### main testing file###
from typing_extensions import TypedDict
from typing import Annotated, Dict, Any
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from langchain_core.messages import AIMessage
import requests
import time
import pyodbc
from contextlib import closing
import pandas as pd
from warnings import filterwarnings
filterwarnings('ignore')
 


In [3]:
class config:
    conn_str = (
        "DRIVER={SQL Server};"
        "SERVER=216.48.191.98;"
        "DATABASE=Agentic_Automation;"
        "UID=ibsadmin;"
        "PWD=Viking@@ibs2023;"
    )

In [4]:
def task_insertion(work_id,webtop_id,process_name,task_name):
    conn=pyodbc.connect(config.conn_str)
    cur=conn.cursor()
    task_master_data=pd.read_sql(f'''select tm.*,td.Dependency_Task_Number from Credit_GPT.dbo.Task_master as tm left join Credit_GPT.dbo.[Task_Dependencies] as td 
                                 on tm.Task_Number=td.Task_Number where tm.Process_Name='{process_name}'
                                  ''',conn)
    task_number_list=[]
    cr=0
    # print(task_master_data)

    while True:
        cr+=1
        if cr==1:
            task_number=task_master_data[task_master_data["Task_Name"]==task_name & task_master_data["Process_Name"] == process_name]["Task_Number"]
            dependecy_task_number=task_master_data[task_master_data["Task_Name"]==task_name & task_master_data["Process_Name"] == process_name]["Dependency_Task_Number"]
        else:
            task_number=task_master_data[task_master_data["Task_Number"].isin(dependecy_task_number)]["Task_Number"]
            dependecy_task_number=task_master_data[task_master_data["Task_Number"].isin(dependecy_task_number)]["Dependency_Task_Number"]

        task_number_list.extend(list(set(task_number)))
        print(f"task_number_list={task_number_list}\ndependecy_task_number={list(dependecy_task_number)}\n==============")
        if dependecy_task_number.empty:
            break
        elif dependecy_task_number.iloc[0] == None and len(dependecy_task_number)==1:
            break
    sTask_number_list="'"+"','".join(task_number_list)+"'"

    Already_present_records=pd.read_sql(f'''select Task_Number from Credit_GPT.dbo.Task_Processing where Webtop_ID='{webtop_id}' and 
                                        Task_Number in ({sTask_number_list})''',conn)
    set_Already_present_records=set(Already_present_records['Task_Number'])
    set_task_number_list=set(task_number_list)
    set_new_tasks_to_insert=set_task_number_list-set_Already_present_records
    sNew_tasks_to_insert="'"+"','".join(set_new_tasks_to_insert)+"'"
    cur.execute(f'''Insert Into Credit_GPT.dbo.Task_Processing 
                    select '{work_id}','{webtop_id}', Task_Number,Task_Name ,'New',getdate(),Null,Null,0,Null,Null,'{process_name}'
                    from Credit_GPT.dbo.Task_Master where Task_Number in ({sNew_tasks_to_insert})
                ''')
    conn.commit()
    conn.close()
    # return task_number_list,set_Already_present_records,set_new_tasks_to_insert,

print(task_insertion('work310','074PZ8600310','Bureau Demog','ReportCreation'))
# 349PZ8633504
# Bureau Analysis


    

TypeError: Cannot perform 'rand_' with a dtyped [object] array and scalar of type [bool]

In [ ]:
# 1) Define the shape of the pipeline state
class State(TypedDict):
    messages: Annotated[list, add_messages]
    data: Dict[str, Any]
 
# 2) Your two Power Automate endpoints
FLOWS = {
    "insertion_in_db": "https://prod-02.centralindia.logic.azure.com:443/workflows/176b9969d13147aa90ae97d9b53c7ca9/triggers/manual/paths/invoke?api-version=2016-06-01&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=cRltkx9lLIr5KRfxpNBvPZcmp2ybs3uxpSM_EvYk42s",
    "doc_download": "https://prod-16.centralindia.logic.azure.com:443/workflows/e8c18012e313410dae07e6ed20ccc816/triggers/manual/paths/invoke?api-version=2016-06-01&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=oMl04_zC72XGN8W-ChrED9jW4pvSs5xUEQQA0BuHs3E"
}
 
def call_flow(flow_name: str, data: dict) -> dict:
    """Trigger a Power Automate flow and report success/failure."""
    try:
        resp = requests.post(FLOWS[flow_name], json=data)
        resp.raise_for_status()
        return {"success": True}
    except Exception as e:
        return {"success": False, "error": str(e)}
 
def insertion_in_db(state: State) -> State:
    result = call_flow("insertion_in_db", state["data"])
    print(f"DB insertion result: {result}")
    return {
        "messages": [
            AIMessage(f"DB insertion: {'✅' if result['success'] else '❌'}")
        ],
        "data": state["data"]
    }
 
def wait_for_status_dbinsertion(state: State) -> State:
    """Poll the workflow_status table until status == 'Completed'."""
    conn_str = (
        "DRIVER={SQL Server};"
        "SERVER=216.48.191.98;"
        "DATABASE=Agentic_Automation;"
        "UID=ibsadmin;"
        "PWD=Viking@@ibs2023;"
    )
    query = """
        SELECT DbInsertion
        FROM [Agentic_Automation].[dbo].[workflow_status]
        WHERE WebtopId = ?
    """
    webtop_id = state["data"]["WebtopId"]
 
    with pyodbc.connect(conn_str, autocommit=True) as conn:
        with closing(conn.cursor()) as cur:
            while True:
                cur.execute(query, webtop_id)
                row = cur.fetchone()
                if row and row[0] == "Completed":
                    break
                time.sleep(5)
    print("DB status: Completed")
    return {
        "messages": [AIMessage("DB status: ✅ Completed")],
        "data": state["data"]
    }
 
def wait_for_status_docdownload(state: State) -> State:
    """Poll the workflow_status table until status == 'Completed'."""
    conn_str = (
        "DRIVER={SQL Server};"
        "SERVER=216.48.191.98;"
        "DATABASE=Agentic_Automation;"
        "UID=ibsadmin;"
        "PWD=Viking@@ibs2023;"
    )
    query = """
        SELECT DocDownload
        FROM [Agentic_Automation].[dbo].[workflow_status]
        WHERE WebtopId = ?
    """
    webtop_id = state["data"]["WebtopId"]
 
    with pyodbc.connect(conn_str, autocommit=True) as conn:
        with closing(conn.cursor()) as cur:
            while True:
                cur.execute(query, webtop_id)
                row = cur.fetchone()
                if row and row[0] == "Completed":
                    break
                time.sleep(5)
    print("Doc download status: Completed")
 
    return {
        "messages": [AIMessage("DB status: ✅ Completed")],
        "data": state["data"]
    }
 
 
 
def doc_download(state: State) -> State:
    result = call_flow("doc_download", state["data"])
    print(f"Document download result: {result}")
    return {
        "messages": [
            AIMessage(f"Document download: {'✅' if result['success'] else '❌'}")
        ],
        "data": {**state["data"], "doc_result": result}
    }
 
def create_pipeline():
    graph = StateGraph(State)
    graph.add_node("insertion_in_db", insertion_in_db)
    graph.add_node("wait_for_status_dbinsertion",  wait_for_status_dbinsertion)
    graph.add_node("doc_download",     doc_download)
    graph.add_node("wait_for_status_docdownload", wait_for_status_docdownload)
 
    graph.set_entry_point("insertion_in_db")
    graph.add_edge("insertion_in_db", "wait_for_status_dbinsertion")
    graph.add_edge("wait_for_status_dbinsertion", "doc_download")
    graph.add_edge("doc_download", "wait_for_status_docdownload")
    graph.add_edge("wait_for_status_docdownload", END)
 
    return graph.compile()
 
if __name__ == "__main__":
    pipeline = create_pipeline()
    # seed the pipeline with an ID of your workflow run
    result = pipeline.invoke({
        "messages": [],
        "data": {"WebtopId": "ABC123"}
    })
    print("Pipeline completed, messages:")
    for msg in result["messages"]:
        print(" •", msg.content)
 
 

In [ ]:
#### main testing file###

from typing_extensions import TypedDict

from typing import Annotated, Dict, Any

from langgraph.graph.message import add_messages

from langgraph.graph import StateGraph, END

from langchain_core.messages import AIMessage

import requests

import time

import pyodbc

from contextlib import closing

import pandas as pd

from warnings import filterwarnings

filterwarnings('ignore')

 
class config:

    conn_str = (

        "DRIVER={SQL Server};"

        "SERVER=216.48.191.98;"

        "DATABASE=Agentic_Automation;"

        "UID=ibsadmin;"

        "PWD=Viking@@ibs2023;"

    )

def task_insertion(work_id,webtop_id,process_name,task_name):

    conn=pyodbc.connect(config.conn_str)

    cur=conn.cursor()

    task_master_data=pd.read_sql(f'''select tm.*,td.Dependency_Task_Number from Credit_GPT.dbo.Task_master as tm left join Credit_GPT.dbo.[Task_Dependencies] as td 

                                 on tm.Task_Number=td.Task_Number where tm.Process_Name='{process_name}'

                                  ''',conn)

    task_number_list=[]

    cr=0

    # print(task_master_data)
 
    while True:

        cr+=1

        if cr==1:

            task_number=task_master_data[task_master_data["Task_Name"]==task_name]["Task_Number"]

            dependecy_task_number=task_master_data[task_master_data["Task_Name"]==task_name]["Dependency_Task_Number"]

        else:

            task_number=task_master_data[task_master_data["Task_Number"].isin(dependecy_task_number)]["Task_Number"]

            dependecy_task_number=task_master_data[task_master_data["Task_Number"].isin(dependecy_task_number)]["Dependency_Task_Number"]
 
        task_number_list.extend(list(set(task_number)))

        print(f"task_number_list={task_number_list}\ndependecy_task_number={list(dependecy_task_number)}\n==============")

        if dependecy_task_number.empty:

            break

        elif dependecy_task_number.iloc[0] == None and len(dependecy_task_number)==1:

            break

    sTask_number_list="'"+"','".join(task_number_list)+"'"
 
    Already_present_records=pd.read_sql(f'''select Task_Number from Credit_GPT.dbo.Task_Processing where Webtop_ID='{webtop_id}' and 

                                        Task_Number in ({sTask_number_list})''',conn)

    set_Already_present_records=set(Already_present_records['Task_Number'])

    set_task_number_list=set(task_number_list)

    set_new_tasks_to_insert=set_task_number_list-set_Already_present_records

    sNew_tasks_to_insert="'"+"','".join(set_new_tasks_to_insert)+"'"

    cur.execute(f'''Insert Into Credit_GPT.dbo.Task_Processing 

                    select '{work_id}','{webtop_id}', Task_Number,Task_Name ,'New',getdate(),Null,Null,0,Null,Null,'{process_name}'

                    from Credit_GPT.dbo.Task_Master where Task_Number in ({sNew_tasks_to_insert})

                ''')
    cur.execute(f'''
                INSERT INTO [Credit_GPT].dbo.Customer_Demog_Detail   (Webtop_ID)
                VALUES ('{webtop_id}')
                INSERT INTO [Credit_GPT].dbo.Verifications_Detail   (Webtop_ID)
                VALUES ('{webtop_id}')
                INSERT INTO [Credit_GPT].dbo.Offer_Detail (Webtop_ID)
                VALUES ('{webtop_id}')
                INSERT INTO [Credit_GPT].dbo.Loan_Detail (Webtop_ID)
                VALUES ('{webtop_id}')
                INSERT INTO [Credit_GPT].dbo.Credit_Headers (Webtop_ID)
                VALUES ('{webtop_id}');
                ''')

    conn.commit()

    conn.close()

    # return task_number_list,set_Already_present_records,set_new_tasks_to_insert,
 
print(task_insertion('12ea4c','074PZ8600310','Bureau Analysis','ReportCreation'))
 
 
    
 

task_number_list=['5']
dependecy_task_number=['4']
task_number_list=['5', '4']
dependecy_task_number=['1', '3']
task_number_list=['5', '4', '3', '1']
dependecy_task_number=[None, '2']
task_number_list=['5', '4', '3', '1', '2']
dependecy_task_number=[None]
None
